In [165]:
# Importation des packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

import seaborn as sns
import re
import json

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    OneHotEncoder, OrdinalEncoder, PolynomialFeatures
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.metrics import (
    mean_absolute_error, root_mean_squared_error, r2_score
)

from sklearn.linear_model import LinearRegression

In [166]:
pd.options.display.max_columns = None #  pour afficher toutes les colonnes de la base
pd.options.display.max_rows = None  # pour afficher toutes les lignes de la base
pd.set_option('display.max_colwidth', None) # on ne définit pas de largeur max des colonnes afin d'éviter de tronquer leur contenu à l'affichage

In [167]:
df_sociodemo = pd.read_csv('./data/train/df_telco_customer_churn_demographics.csv', sep=',')
df_sociodemo.head()
print(f"Le df initial contenant ls données socio_demographiques contient {df_sociodemo.shape[0]} lignes et {df_sociodemo.shape[1]} variables.")

df_services = pd.read_csv('./data/train/df_telco_customer_churn_services.csv', sep=',')
df_services.head()
print(f"Le df initial contenant ls services contient {df_services.shape[0]} lignes et {df_services.shape[1]} variables.")

df_churn = pd.read_csv('./data/train/df_telco_customer_churn_status.csv', sep=',')
df_churn.head()
print(f"Le df initial contenant les status des clients contient {df_churn.shape[0]} lignes et {df_churn.shape[1]} variables.")

Le df initial contenant ls données socio_demographiques contient 5158 lignes et 8 variables.
Le df initial contenant ls services contient 5158 lignes et 28 variables.
Le df initial contenant les status des clients contient 5158 lignes et 2 variables.


In [168]:
df_sociodemo.sort_values(by="customer_id", inplace = True)
df_sociodemo.head()
df_churn.sort_values(by="customer_id", inplace = True)
df_churn.head()
df_services.sort_values(by="customer_id", inplace = True)
df_services.head()

,customer_id,referred_a_friend,number_of_referrals,tenure_in_months,offer,phone_service,avg_monthly_long_distance_charges,multiple_lines,internet_service,internet_type,avg_monthly_gb_download,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,contract,paperless_billing,payment_method,monthly_charge,total_charges,total_refunds,total_extra_data_charges,total_long_distance_charges,total_revenue
2917,0002-ORFBO,Yes,2,9,NaN,Yes,42.39,No,Yes,Cable,16,No,Yes,No,Yes,Yes,No,No,Yes,One Year,Yes,Credit Card,65.6,593.30,0.00,0,381.51,974.81
3502,0003-MKNFE,No,0,9,NaN,Yes,10.69,Yes,Yes,Cable,10,No,No,No,No,No,Yes,Yes,No,Month-to-Month,No,Credit Card,59.9,542.40,38.33,10,96.21,610.28
290,0011-IGKFF,Yes,1,13,Offer D,Yes,27.82,No,Yes,Fiber Optic,4,No,Yes,Yes,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51
265,0013-EXCHZ,Yes,3,3,NaN,Yes,7.38,No,Yes,Fiber Optic,11,No,No,No,Yes,Yes,No,No,Yes,Month-to-Month,Yes,Credit Card,83.9,267.40,0.00,0,22.14,289.54
2407,0013-MHZWF,No,0,9,Offer E,Yes,16.77,No,Yes,Cable,73,No,No,No,Yes,Yes,Yes,Yes,Yes,Month-to-Month,Yes,Credit Card,69.4,571.45,0.00,0,150.93,722.38


In [169]:
df_services.head()

,customer_id,referred_a_friend,number_of_referrals,tenure_in_months,offer,phone_service,avg_monthly_long_distance_charges,multiple_lines,internet_service,internet_type,avg_monthly_gb_download,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,contract,paperless_billing,payment_method,monthly_charge,total_charges,total_refunds,total_extra_data_charges,total_long_distance_charges,total_revenue
2917,0002-ORFBO,Yes,2,9,NaN,Yes,42.39,No,Yes,Cable,16,No,Yes,No,Yes,Yes,No,No,Yes,One Year,Yes,Credit Card,65.6,593.30,0.00,0,381.51,974.81
3502,0003-MKNFE,No,0,9,NaN,Yes,10.69,Yes,Yes,Cable,10,No,No,No,No,No,Yes,Yes,No,Month-to-Month,No,Credit Card,59.9,542.40,38.33,10,96.21,610.28
290,0011-IGKFF,Yes,1,13,Offer D,Yes,27.82,No,Yes,Fiber Optic,4,No,Yes,Yes,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51
265,0013-EXCHZ,Yes,3,3,NaN,Yes,7.38,No,Yes,Fiber Optic,11,No,No,No,Yes,Yes,No,No,Yes,Month-to-Month,Yes,Credit Card,83.9,267.40,0.00,0,22.14,289.54
2407,0013-MHZWF,No,0,9,Offer E,Yes,16.77,No,Yes,Cable,73,No,No,No,Yes,Yes,Yes,Yes,Yes,Month-to-Month,Yes,Credit Card,69.4,571.45,0.00,0,150.93,722.38


In [170]:
df = pd.merge(pd.merge(df_sociodemo, df_services, how='outer', on='customer_id',  validate='one_to_one', sort=False
), df_churn, how='outer', on='customer_id',  validate='one_to_one', sort=False)
print(f"Le df initial contenant les status des clients contient {df.shape[0]} lignes et {df.shape[1]} variables.")

# Le fait de faire un merge outer vient aggreger tous les id des clients, s'il y a des doublons au niveau de l'ID alors validate signalera une erreur

Le df initial contenant les status des clients contient 5158 lignes et 36 variables.


In [171]:
df.info()
df.to_csv('./data/train/df_telco.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5158 entries, 0 to 5157
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   customer_id                        5158 non-null   object 
 1   gender                             5158 non-null   object 
 2   age                                5158 non-null   int64  
 3   under_30                           5158 non-null   object 
 4   senior_citizen                     5158 non-null   object 
 5   married                            5158 non-null   object 
 6   dependents                         5158 non-null   object 
 7   number_of_dependents               5158 non-null   int64  
 8   referred_a_friend                  5158 non-null   object 
 9   number_of_referrals                5158 non-null   int64  
 10  tenure_in_months                   5158 non-null   int64  
 11  offer                              2310 non-null   objec

In [172]:
# Affichage des variables qualitatives :
print(f"Le dataFrame comprend : \n{df.select_dtypes(exclude = 'object').columns.size} variables qualitatives  : \n{df.select_dtypes(include='object').columns.tolist()}")

# Affichage des variables quantitatives :
print(f"{df.select_dtypes(include='object').columns.size} variables quantitatives : \n{df.select_dtypes(exclude='object').columns.tolist()}")

Le dataFrame comprend : 
13 variables qualitatives  : 
['customer_id', 'gender', 'under_30', 'senior_citizen', 'married', 'dependents', 'referred_a_friend', 'offer', 'phone_service', 'multiple_lines', 'internet_service', 'internet_type', 'online_security', 'online_backup', 'device_protection_plan', 'premium_tech_support', 'streaming_tv', 'streaming_movies', 'streaming_music', 'unlimited_data', 'contract', 'paperless_billing', 'payment_method']
23 variables quantitatives : 
['age', 'number_of_dependents', 'number_of_referrals', 'tenure_in_months', 'avg_monthly_long_distance_charges', 'avg_monthly_gb_download', 'monthly_charge', 'total_charges', 'total_refunds', 'total_extra_data_charges', 'total_long_distance_charges', 'total_revenue', 'churn_value']


In [173]:
cat_cols = df.select_dtypes(include=object).columns.tolist()
num_cols = df.select_dtypes(exclude=object).columns.tolist()
all_cols = df.columns.tolist()

In [174]:
for var in cat_cols :
    print(f"La variable catégorielle '{var}' présente {df[var].nunique()} catégories")
    if df[var].nunique() < 10 : 
        print(f"{df[var].unique()}")
    print("\n")

La variable catégorielle 'customer_id' présente 5158 catégories


La variable catégorielle 'gender' présente 2 catégories
['Female' 'Male']


La variable catégorielle 'under_30' présente 2 catégories
['No' 'Yes']


La variable catégorielle 'senior_citizen' présente 2 catégories
['No' 'Yes']


La variable catégorielle 'married' présente 2 catégories
['Yes' 'No']


La variable catégorielle 'dependents' présente 2 catégories
['No' 'Yes']


La variable catégorielle 'referred_a_friend' présente 2 catégories
['Yes' 'No']


La variable catégorielle 'offer' présente 5 catégories
[nan 'Offer D' 'Offer E' 'Offer A' 'Offer B' 'Offer C']


La variable catégorielle 'phone_service' présente 2 catégories
['Yes' 'No']


La variable catégorielle 'multiple_lines' présente 2 catégories
['No' 'Yes']


La variable catégorielle 'internet_service' présente 2 catégories
['Yes' 'No']


La variable catégorielle 'internet_type' présente 3 catégories
['Cable' 'Fiber Optic' 'DSL' nan]


La variable catégorielle 'o

Analyse de code de départements :

In [199]:
df["test"] = df["customer_id"].apply(lambda x : x.split("-")[1])

In [204]:
mask  = df["test"]== "CYWMH"
df[mask]

,customer_id,gender,age,under_30,senior_citizen,married,dependents,number_of_dependents,referred_a_friend,number_of_referrals,tenure_in_months,offer,phone_service,avg_monthly_long_distance_charges,multiple_lines,internet_service,internet_type,avg_monthly_gb_download,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,contract,paperless_billing,payment_method,monthly_charge,total_charges,total_refunds,total_extra_data_charges,total_long_distance_charges,total_revenue,churn_value,monthly_charge_slo,test
730,1431-CYWMH,Female,35,No,No,Yes,Yes,1,No,0,22,Offer D,Yes,46.45,No,No,NaN,0,No,No,No,No,No,No,No,No,Month-to-Month,No,Bank Withdrawal,19.05,454.05,0.0,0,1021.90,1475.95,0,20.638636,CYWMH
2272,4462-CYWMH,Male,65,No,Yes,Yes,No,0,Yes,7,62,Offer B,Yes,27.49,Yes,Yes,Fiber Optic,3,No,Yes,No,No,No,Yes,Yes,Yes,Month-to-Month,Yes,Bank Withdrawal,89.80,5629.55,0.0,0,1704.38,7333.93,0,90.799194,CYWMH


Cl : les codes pays ne servent à rien et n'ont pas à être inclus dans l'étude

Controle des doublons

In [175]:
df.duplicated().sum()

0

Controle des valeurs manquantes :

In [176]:
counter  = 0 # Initialisation d'un compteur des colonnes ayant des valeurs manquantes

# Nous itérons sur toutes les variables quantitatives :
for var in num_cols :
    # Si une colonne présente des valeurs manquantes:
    #  - on affiche son nom et le nombre de valeurs manquantes
    #  - on incrémente le compteur

    if (df[var].isna().sum() > 0) :
        print(f"{var} : {df[var].isna().sum()} manquantes.")
        counter += 1

print(f"{counter} variables quantitatives présentent des valeurs manquantes.")

# Nous itérons sur toutes les variables quantitatives :
for var in cat_cols :
    # Si une colonne présente des valeurs manquantes:
    #  - on affiche son nom et le nombre de valeurs manquantes
    #  - on incrémente le compteur

    if (df[var].isna().sum() > 0) :
        print(f"{var} : {df[var].isna().sum()} manquantes.")
        counter += 1

print(f"{counter} variables qualitatives présentent des valeurs manquantes.")

0 variables quantitatives présentent des valeurs manquantes.
offer : 2848 manquantes.
internet_type : 1107 manquantes.
2 variables qualitatives présentent des valeurs manquantes.


Additional checks :
** if age >= 65 --> senior_citizen = yes
** if age <= 30 --> under_30 = yes
** if referred_a_friend --> number_of_referrals > 0
** check si le conjoint fait parti des dépendants
** check si dependents = yes number_of_dependants > 0

In [177]:
#** if age >= 65 senior_citizen
((df["age"]>= 65) & (df["senior_citizen"]=="No")).sum()


0

In [178]:
#** if age <= 30 under_30
((df["age"]< 30) & (df["under_30"]=="No")).sum()

0

In [179]:
# if referred_a_friend --> number_of_referrals > 0
((df["referred_a_friend"]== "yes") & (df["number_of_referrals"]<1)).sum()

mask = df["referred_a_friend"]== "No"
df[mask]["number_of_referrals"].describe()

count    2827.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: number_of_referrals, dtype: float64

In [180]:
# check if le conjoint fait parti des dépendants
mask = df["married"] =="Yes"
df[mask]["number_of_dependents"].describe()


count    2460.000000
mean        0.785772
std         1.132912
min         0.000000
25%         0.000000
50%         0.000000
75%         2.000000
max         9.000000
Name: number_of_dependents, dtype: float64

In [181]:
mask = (df["married"] =="Yes") & (df["number_of_dependents"]<1)
mask.sum()
## cl :le conjoint ne fait ps parti des dépendants

1504

In [182]:
#check si dependents = yes number_of_dependants > 0
mask = (df["dependents"] == "Yes") & (df["number_of_dependents"]<1)
mask.sum()

0

Cohérence dans les services

In [183]:
# Vefification que si pas d'internet service alors le type internet est Nan
mask = (df["internet_service"] == "No") & (df["internet_type"].isna())
mask.sum()

1107

In [184]:
mask = (df["internet_service"] == "Yes") 
print(df[mask]["internet_type"].value_counts())
print(mask.sum())

## Cl : Ok

internet_type
Fiber Optic    2229
DSL            1222
Cable           600
Name: count, dtype: int64
4051


In [185]:
# Vérification que total_charges / tenure in month = monthly_charge
df["monthly_charge_slo"] = df["total_charges"]/df["tenure_in_months"]
(df["monthly_charge"]-df["monthly_charge_slo"]).describe()

count    5158.000000
mean        0.007242
std         2.662022
min       -17.925000
25%        -1.176562
50%         0.000000
75%         1.160768
max        19.125000
dtype: float64

In [186]:
df["tenure_in_months"].describe()

count    5158.000000
mean       32.205312
std        24.571588
min         1.000000
25%         9.000000
50%        29.000000
75%        55.000000
max        72.000000
Name: tenure_in_months, dtype: float64

In [187]:
cols = ["tenure_in_months", "monthly_charge", "total_charges", "total_extra_data_charges", "total_long_distance_charges", "total_refunds", 'total_revenue']
a = df[cols]
a.head( n =20)
a["monmoncha"] = a.loc[:,"tenure_in_months"]*a.loc[:,"monthly_charge"]

/tmp/ipykernel_7688/4134417317.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a["monmoncha"] = a.loc[:,"tenure_in_months"]*a.loc[:,"monthly_charge"]


In [188]:
a["total_revenue_check"] = a.loc[:,"total_charges"]-a.loc[:,"total_refunds"]+a.loc[:,"total_extra_data_charges"]+a.loc[:,"total_long_distance_charges"]
(a["total_revenue_check"]-a["total_revenue"]).describe()

/tmp/ipykernel_7688/3376738245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a["total_revenue_check"] = a.loc[:,"total_charges"]-a.loc[:,"total_refunds"]+a.loc[:,"total_extra_data_charges"]+a.loc[:,"total_long_distance_charges"]


count    5.158000e+03
mean    -4.112001e-16
std      2.887920e-13
min     -1.818989e-12
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.818989e-12
dtype: float64

CL : le total_revenue = total_charges + total_extra_data_charges + total_long_distance_charges - total_refunds


Detail des différentes souscription :
Offre A : 

In [189]:
col_to_look =[ "device_protection_plan"  , "internet_service" , "multiple_lines" , "online_backup" , "online_security", "phone_service", "premium_tech_support", "unlimited_data"]


In [190]:
df["offer"].unique()

array([nan, 'Offer D', 'Offer E', 'Offer A', 'Offer B', 'Offer C'],
      dtype=object)

In [191]:
for offre in df["offer"].unique() : 
    #Offre A
    mask  = df["offer"]==offre
    print(f"Il y a {mask.sum()} clients ayant souscrit à {offre}.")

    for var in col_to_look : 
        print(df[mask][var].value_counts())


Il y a 0 clients ayant souscrit à nan.
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
Il y a 445 clients ayant souscrit à Offer D.
device_protection_plan
No     338
Yes    107
Name: count, dtype: int64
internet_service
Yes    332
No     113
Name: count, dtype: int64
multiple_lines
No     292
Yes    153
Name: count, dtype: int64
online_backup
No     340
Yes    105
Name: count, dtype: int64
online_security
No     358
Yes     87
Name: count, dtype: int64
phone_service
Yes    408
No      37
Name: count, dtype: int64
premium_tech_support
No     358
Yes     87
Name: count, dtype: int64
unlimited_data
Yes    283
No     162
Name: count, dtype: int64
Il y a 597 clients ayant souscrit à Offer E.
device_protection_plan
No     509
Yes     88
Nam

In [192]:
#Offre A
mask  = df["offer"].isna()
print(f"Il y a {mask.sum()} clients ayant souscrit à {offre}.")

for var in col_to_look : 
    print(df[mask][var].value_counts())


Il y a 2848 clients ayant souscrit à Offer C.
device_protection_plan
No     1915
Yes     933
Name: count, dtype: int64
internet_service
Yes    2215
No      633
Name: count, dtype: int64
multiple_lines
No     1727
Yes    1121
Name: count, dtype: int64
online_backup
No     1916
Yes     932
Name: count, dtype: int64
online_security
No     2063
Yes     785
Name: count, dtype: int64
phone_service
Yes    2568
No      280
Name: count, dtype: int64
premium_tech_support
No     2040
Yes     808
Name: count, dtype: int64
unlimited_data
Yes    1888
No      960
Name: count, dtype: int64


Cl : Il n'y a pas de pack incluant les différents services supplemntaires dans les différentes offres proposés

Rapport entre unimited data & avg_monthly_gb_dosnload, online_backup, streaming_movies, treaming_music, streaming_tv

In [193]:
col_unlimited_data= [ "streaming_movies","streaming_music","streaming_tv","online_backup"]

In [194]:
mask = df["unlimited_data"] == "Yes"
print(f"Il y a {mask.sum()} clients ayant un forfait illimité")

for var in col_unlimited_data : 
    print(df[mask][var].value_counts())

print(df[mask]["avg_monthly_gb_download"].describe())
print(df["avg_monthly_gb_download"].describe())

Il y a 3473 clients ayant un forfait illimité
streaming_movies
No     1767
Yes    1706
Name: count, dtype: int64
streaming_music
No     1925
Yes    1548
Name: count, dtype: int64
streaming_tv
No     1762
Yes    1711
Name: count, dtype: int64
online_backup
No     1951
Yes    1522
Name: count, dtype: int64
count    3473.000000
mean       26.286784
std        19.629007
min         2.000000
25%        13.000000
50%        21.000000
75%        30.000000
max        85.000000
Name: avg_monthly_gb_download, dtype: float64
count    5158.000000
mean       20.671578
std        20.506272
min         0.000000
25%         4.000000
50%        17.000000
75%        27.000000
max        85.000000
Name: avg_monthly_gb_download, dtype: float64


Cl :il n'y a pas de relation claire entre la souscription d'un forfait illimté et les streaming video, music + tv + online_back_up

On regarde le lien entre total_extra_data_charges et unlimited_data
 et

In [195]:
df[df["unlimited_data"] == "Yes"]["total_extra_data_charges"].describe()

count    3473.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: total_extra_data_charges, dtype: float64

In [196]:
df[df["unlimited_data"] == "No"]["total_extra_data_charges"].describe()

count    1685.000000
mean       20.949555
std        40.247753
min         0.000000
25%         0.000000
50%         0.000000
75%        20.000000
max       150.000000
Name: total_extra_data_charges, dtype: float64

Cl :Il semble que les clients souscrivant un abo cunlimited_data n'ont jamais avant été en dépassement.
Alors qu'il arrive que les autres soient en dépassement.

Déséquilibre au sein de la target _ evaluation

In [197]:
df["churn_value"].value_counts()

churn_value
0    3799
1    1359
Name: count, dtype: int64